In [1]:
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv("../commonlit-evaluate-student-summaries/summaries_train.csv")

In [3]:
data.prompt_id.value_counts()

39c16e    2057
3b9047    2009
ebad26    1996
814d6b    1103
Name: prompt_id, dtype: int64

In [6]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADE17\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADE17\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [11]:
def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    
    tokens = [token.lower() for token in tokens]
    
    tokens = [token for token in tokens if token.isalnum()]
    
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

def extract_features(texts):
    tfidf_vectorizer = TfidfVectorizer()
    
    features = tfidf_vectorizer.fit_transform(texts)
    
    return features

In [12]:
text_summaries = data.text

In [13]:
preprocessed_summaries = [preprocess_text(summary) for summary in text_summaries]

tfidf_features = extract_features(preprocessed_summaries)

In [19]:
target_labels = data[['content', 'wording']]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_features, target_labels, test_size=0.2, random_state=42)

In [21]:
from xgboost import XGBRegressor
regressor = XGBRegressor()

In [22]:
multi_output_regressor = MultiOutputRegressor(regressor)

In [23]:
multi_output_regressor.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...))

In [24]:
predictions = multi_output_regressor.predict(X_test)

In [26]:
predictions

array([[ 0.07724323,  0.08839147],
       [-0.35684004, -0.07453288],
       [-0.5821281 , -0.49445426],
       ...,
       [-0.2782058 , -0.39096385],
       [ 0.9471949 ,  0.5582533 ],
       [-0.9299958 , -1.2015647 ]], dtype=float32)

In [27]:
mse = mean_squared_error(y_test, predictions, multioutput='raw_values')

In [28]:
mse

array([0.32043943, 0.48186885])

In [29]:
mse.mean()

0.40115413955378276